In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import ImageFolder

#this is a test 
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!pip install opendatasets --upgrade --quiet
import opendatasets as od

In [ ]:
dataset_url = 'https://www.kaggle.com/datasets/aneesh10/cricket-shot-dataset'

In [ ]:
od.download(dataset_url)

Skipping, found downloaded files in "./cricket-shot-dataset" (use force=True to force download)


In [ ]:
data_dir = './cricket-shot-dataset/data'
import os

In [ ]:
os.listdir(data_dir)

['drive', 'pullshot', 'sweep', 'legglance-flick']

In [ ]:
dataset = ImageFolder(data_dir)

In [ ]:
import torchvision.transforms as tt
# normalize = tt.Normalize(
#         mean=[0.4914, 0.4822, 0.4465],
#         std=[0.2023, 0.1994, 0.2010],
# )

dataset = ImageFolder(data_dir, tt.Compose([tt.Resize(64), 
                                            tt.RandomCrop(64), 
                                            tt.ToTensor()]))

In [ ]:
val_pct = 0.5
val_size = int(val_pct * len(dataset))
train_size = len(dataset) - val_size

train_size, val_size

(2362, 2362)

In [ ]:
from torch.utils.data import random_split

train_ds, valid_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(valid_ds)

(2362, 2362)

In [ ]:
mean = 0.0
for images, _ in train_ds: #image dim: 128*3*64*64
    # batch_samples = images.size(0) # get batch size
    images = images.view(images.size(0), -1) # reshape to 128*3*4096
    mean += images.mean(1) # mean by collapsing dimension 2 then sum over dimension 1
mean = mean / len(train_ds) # divide by total dataset as we only divided by 4096 and not 128 also

var = 0.0
for images, _ in train_ds:
    # batch_samples = images.size(0)
    images = images.view(images.size(0), -1)
    var += ((images - mean.unsqueeze(1))**2).mean(1)
std = torch.sqrt(var / (len(train_ds)))

In [ ]:
print(mean)
print(std)

tensor([0.4965, 0.5012, 0.4537])
tensor([0.2711, 0.2659, 0.2671])


In [ ]:
train_ds.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=mean,std=std)
            ])
valid_ds.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=mean,std=std)
            ])

In [ ]:
from torch.utils.data import DataLoader

batch_size = 128

train_dl = DataLoader(train_ds, 
                      batch_size, 
                      shuffle=True, 
                      num_workers=4, 
                      pin_memory=True)

valid_dl = DataLoader(valid_ds, 
                    batch_size, 
                    num_workers=4, 
                    pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        # Input: 128 x 3 x 64 x 64
        self.conv1 = conv_block(in_channels, 64) # 128 x 64 x 64 x 64
        self.conv2 = conv_block(64, 128, pool=True) # 128 x 128 x 32 x 32
        self.res1 = nn.Sequential(conv_block(128, 128), # 128 x 128 x 32 x 32
                                  conv_block(128, 128)) # 128 x 128 x 32 x 32
        
        self.conv3 = conv_block(128, 256, pool=True) # 128 x 256 x 16 x 16
        self.conv4 = conv_block(256, 512, pool=True) # 128 x 512 x 8 x 8 
        self.res2 = nn.Sequential(conv_block(512, 512), # 128 x 512 x 8 x 8 
                                  conv_block(512, 512)) # 128 x 512 x 8 x 8 
        
        self.classifier = nn.Sequential(nn.AdaptiveMaxPool2d(1), # 128 x 512 x 1 x 1 
                                        nn.Flatten(), # 128 x 512
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [ ]:
num_classes = 4
num_epochs = 100
batch_size = 16
learning_rate = 0.01
in_channels = 3
model = ResNet9(3,4).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.9, patience=5, threshold=0.0001, eps=1e-08, verbose=True)


# Train the model
total_step = len(train_dl)

In [ ]:
total_step = len(train_dl)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dl):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_dl:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        val_acc = 100 * correct / total
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, val_acc))
    scheduler.step(val_acc)

Epoch [1/100], Step [19/19], Loss: 8.7101
Accuracy of the network on the 5000 validation images: 28.45046570702794 %
Epoch [2/100], Step [19/19], Loss: 1.9005
Accuracy of the network on the 5000 validation images: 28.831498729889923 %
Epoch [3/100], Step [19/19], Loss: 1.9171
Accuracy of the network on the 5000 validation images: 28.535139712108382 %
Epoch [4/100], Step [19/19], Loss: 1.3836
Accuracy of the network on the 5000 validation images: 27.98475867908552 %
Epoch [5/100], Step [19/19], Loss: 2.4458
Accuracy of the network on the 5000 validation images: 31.456392887383572 %
Epoch [6/100], Step [19/19], Loss: 1.1886
Accuracy of the network on the 5000 validation images: 30.48264182895851 %
Epoch [7/100], Step [19/19], Loss: 2.0653
Accuracy of the network on the 5000 validation images: 35.732430143945805 %
Epoch [8/100], Step [19/19], Loss: 2.1363
Accuracy of the network on the 5000 validation images: 34.843353090601184 %
Epoch [9/100], Step [19/19], Loss: 1.2009
Accuracy of the n